<a href="https://colab.research.google.com/github/eslambasem99/Recommendation-System/blob/main/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/MyDrive/Coding/Eslam/recommedtion_system"

/content/gdrive/MyDrive/Coding/Eslam/recommedtion_system


In [ ]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer

# Capture similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
data = pd.read_csv("data/netflix_titles.csv")
data.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [ ]:
data.dropna(subset=['cast','title','description','listed_in'],inplace=True,axis=0)
data = data.reset_index(drop=True)

In [ ]:
data['listed_in'] = [re.sub(r'[^\w\s]', '', t) for t in data['listed_in']]
data['cast'] = [re.sub(',',' ',re.sub(' ','',t)) for t in data['cast']]
data['description'] = [re.sub(r'[^\w\s]', '', t) for t in data['description']]
data['title'] = [re.sub(r'[^\w\s]', '', t) for t in data['title']]

In [ ]:
data["combined"] = data['listed_in'] + '  ' + data['cast'] + ' ' + data['title'] + ' ' + data['description']
data.drop(['listed_in','cast','description'],axis=1,inplace=True)
data.head()

,show_id,type,title,director,country,date_added,release_year,rating,duration,combined
0,s2,TV Show,Blood Water,NaN,South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows TV Dramas TV Mysteries ...
1,s3,TV Show,Ganglands,Julien Leclercq,NaN,"September 24, 2021",2021,TV-MA,1 Season,Crime TV Shows International TV Shows TV Actio...
2,s5,TV Show,Kota Factory,NaN,India,"September 24, 2021",2021,TV-MA,2 Seasons,International TV Shows Romantic TV Shows TV Co...
3,s6,TV Show,Midnight Mass,Mike Flanagan,NaN,"September 24, 2021",2021,TV-MA,1 Season,TV Dramas TV Horror TV Mysteries KateSiegel Z...
4,s7,Movie,My Little Pony A New Generation,"Robert Cullen, José Luis Ucha",NaN,"September 24, 2021",2021,PG,91 min,Children Family Movies VanessaHudgens Kimiko...


In [ ]:
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(data["combined"])
cosine_similarities = linear_kernel(matrix,matrix)
movie_title = data['title']
indices = pd.Series(data.index, index=data['title'])

In [ ]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    return movie_title.iloc[movie_indices]

In [ ]:
content_recommender('The Crown')


,title
709,Downton Abbey
543,The Queen
5283,Witches A Century of Murder
6665,London Spy
6848,My Hotter Half
4797,Reign
7457,The Blue Planet A Natural History of the Oceans
7689,The Real Football Factories
4528,Collateral
4625,Lovesick


In [ ]:
title = 'The Crown'
suggestions = content_recommender(title)

suggestions_df = pd.DataFrame(data=suggestions)

In [ ]:
suggestions_df

,title
709,Downton Abbey
543,The Queen
5283,Witches A Century of Murder
6665,London Spy
6848,My Hotter Half
4797,Reign
7457,The Blue Planet A Natural History of the Oceans
7689,The Real Football Factories
4528,Collateral
4625,Lovesick
